# 機械学習

## 目次

- 項目の説明
- 欠損値の扱い
- カテゴリー変数の扱い

## Section2 データの解説

In [ ]:
# # Googleドライブのマウント（Colab使いのみ）

# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/dlc/week1

In [ ]:
# matplotが使えるようにする

import matplotlib
%matplotlib inline

In [ ]:
# データのロード

import pandas as pd

data = pd.read_csv("./data/train.csv")

### 2.1 項目の説明

In [ ]:
# 先頭3行を表示

data.head(3)

- PassengerId – 乗客識別ユニークID
- Survived – 生存フラグ（0=死亡、1=生存）　←　これを予測する
- Pclass – チケットクラス（1st, 2nd, 3rd）
- Name – 乗客の名前
- Sex – 性別（male=男性、female＝女性）
- Age – 年齢
- SibSp – タイタニックに同乗している兄弟/配偶者の数
- Parch – タイタニックに同乗している親/子供の数
- Ticket – チケット番号
- Fare – 料金
- Cabin – 客室番号
- Embarked – タイタニックへ乗った港（C=Cherbourg, S=Southampton, Q=Queenstown）

In [ ]:
# データのサイズを確認

data.shape

In [ ]:
# データの欠損・型を確認

data.info()

In [ ]:
# 欠損率を確認

def nullCountFig(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    counted_table = pd.concat([null_val, percent], axis=1)
    counted_figure = counted_table.rename(
        columns = {0 : '欠損数', 1 : '欠損率(%)'}
    )
    return counted_figure

nullCountFig(data)   

### 2.2 欠損値の扱い

欠損には大きく3種類あります。

- 完全にランダムに欠損
- 観測データに依存する欠損（特定のデータのみ欠損）
- 欠損データに依存する欠損（そもそもデータとして収集されていない）

これらを考慮した上で、欠損値に対応する主な方針は4つです。

1. 欠損のある行、列を除外する
2. 何らかの値で埋める（単変量補完】
3. 何らかの値で埋める（多変量補完）
4. 欠損値を受け入れてくれるモデルを使う

#### 1. 欠損のある行、列を除外する

メリット：かんたん

デメリット：予測性能の低下を招きやすい

In [ ]:
# 特定のデータを除外する

non_null_data1 = data.drop(['Age', 'Cabin', 'Embarked'], axis=1)
display(non_null_data1)

In [ ]:
# 特定のデータを取り出す

non_null_data2 = data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare']]
display(non_null_data2)

In [ ]:
# 欠損値が一つでも含まれる列を除外する

non_null_data3 = data.dropna()
display(non_null_data3)

#### 2. 何らかの値で埋める（単変量補完）

メリット：かんたん

デメリット：欠損が多いと効果薄

In [ ]:
# 共通の値（0）で埋める

simple_filled_data1 = data.fillna(0)
display(simple_filled_data1)

In [ ]:
# 平均値で埋める

## 平均値：.mean()
## 中央値：.median()
## 最瀕値：.mode()
simple_filled_data2 = data.fillna(data.mean())
display(simple_filled_data2)

In [ ]:
# 直前or直後の値で埋める（時系列データ向き）

## 直前：method='ffill'
## 直後：method='bfill'
simple_filled_data3 = data.fillna(method = 'ffill')
display(simple_filled_data3)

#### 3. 何らかの値で埋める（多変量補完）

メリット：予測精度が高くなりがち

デメリット：うまくいかなかった時、原因がわからないがち

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt

# 数値だけのデータを作る
only_num_data = data.drop(['PassengerId', 'Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1)
only_num_data_columns = only_num_data.columns

# 散布図で確認するための関数
def figmake(df, key):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))
    df.plot.scatter(x=key, y='Age', ax=axes[0])
    only_num_data.plot.scatter(x=key, y='Age', ax=axes[1])
    plt.show()

In [ ]:
# ベイジアンブリッジを使う

bayesian = pd.DataFrame(
                IterativeImputer().fit_transform(only_num_data)
                , columns=only_num_data_columns
            )

display(bayesian)
figmake(bayesian, 'Pclass')

In [ ]:
# ランダムフォレストを使う

from sklearn.ensemble import RandomForestRegressor

randf = pd.DataFrame(
            IterativeImputer(RandomForestRegressor()).fit_transform(only_num_data)
            , columns=only_num_data_columns
        )

display(randf)
figmake(randf, 'Pclass')

In [ ]:
# KNNを使う

from sklearn.impute import KNNImputer

knn = pd.DataFrame(
            KNNImputer(n_neighbors=2).fit_transform(only_num_data)
            , columns=only_num_data_columns
        )

display(knn)
figmake(knn, 'Pclass')

#### 4. 欠損値を受け入れてくれるモデルを使う

メリット：予測精度が高くなりがち

デメリット：ちょっとめんどくさい

XGBoost, LightGBMが有名

### 2.3 カテゴリー変数の扱い

文字列のままでは基本的にモデルへ入力することができないため、代わりとなる数値に変換する必要があります。

一般的に文字列データは量的ではなく質的なデータなため、扱いにくい場合があります。

そこで、[A,B,C]のような複数の値のある要素をAの有無という形式に変換することで扱いやすくします。この処理を **One-Hot エンコーディング** といいます。

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 乗船港のみのデータ作成
embarked_data = data['Embarked'].dropna()

# エンコーダの定義
encoder = OneHotEncoder(sparse=False)

display(
    pd.DataFrame(
        encoder.fit_transform(embarked_data.values.reshape(-1, 1))
        ,columns=encoder.categories_
    )
)